In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [2]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
#print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from config.config import *


c:/Users/Admin/Data/ads_fraud_detection


In [3]:
import torch
#print(torch.version.cuda)  # Hiển thị phiên bản CUDA mà PyTorch đang sử dụng

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on gpu.")

CUDA is not available. Training on gpu.


In [5]:
save_dir=f"{exps_dir}/exp1/exp_smote"
if os.path.exists(save_dir) == False: 
  os.makedirs(save_dir, exist_ok=True)



test_size=0.33
seed=42

In [6]:
# feature_selection=dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']
# feature_selection = {key: value for key, value in feature_selection.item().items()}
# feature_selection

* kiểm tra và tạo các thư mục (nếu chưa có)
* tập test 33%

# 5. Xây dựng và đánh giá mô hình học sâu Neutual Network

In [7]:
# bg_model=joblib.load(f'{save_dir}/bg_model.joblib')
# rf_model=joblib.load(f'{save_dir}/rf_model.joblib')
# svm_model=joblib.load(f'{save_dir}/svm_model.joblib')

In [8]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{save_dir}/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 0.6555772994129159, 1: 2.106918238993711}

In [9]:
device = torch.device("cpu")


In [10]:
X_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values.squeeze(), dtype=torch.long)
X_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values.squeeze(), dtype=torch.long)

In [11]:
# num_classes = len(np.unique(y_train))
# #print(f"Số lớp: {num_classes}")

In [12]:
# # y_train_onehot = torch.nn.functional.one_hot(y_train, num_classes=2)
# # y_test_onehot = torch.nn.functional.one_hot(y_test, num_classes=2)

# #print("Mã hóa one-hot của y_train:\n", y_train_onehot)
# #print("Mã hóa one-hot của y_test:\n", y_test_onehot)

In [13]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data.values, dtype=torch.float32)  # Chuyển features thành tensor
        self.y_data = y_data  # Nhãn đã được mã hóa one-hot

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [14]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
class_weights = torch.tensor([class_weights_dict[key] for key in sorted(class_weights_dict.keys())], dtype=torch.float32)
class_weights = class_weights
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [16]:
x_test.shape

(330, 39)

In [17]:
x_train.shape

(1022, 39)

In [18]:
tabnet_params = {
    "n_d": 16,
    "n_a": 16,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 8.80416 | val_0_unsup_loss_numpy: 59.052978515625|  0:00:00s
epoch 1  | loss: 6.14867 | val_0_unsup_loss_numpy: 36.15195846557617|  0:00:00s
epoch 2  | loss: 4.86443 | val_0_unsup_loss_numpy: 10.468839645385742|  0:00:00s
epoch 3  | loss: 4.13779 | val_0_unsup_loss_numpy: 9.760029792785645|  0:00:00s
epoch 4  | loss: 3.50821 | val_0_unsup_loss_numpy: 5.360750198364258|  0:00:00s
epoch 5  | loss: 3.075   | val_0_unsup_loss_numpy: 3.9730899333953857|  0:00:00s
epoch 6  | loss: 2.70275 | val_0_unsup_loss_numpy: 3.4019598960876465|  0:00:00s
epoch 7  | loss: 2.4362  | val_0_unsup_loss_numpy: 2.7136099338531494|  0:00:00s
epoch 8  | loss: 2.19751 | val_0_unsup_loss_numpy: 2.245879888534546|  0:00:00s
epoch 9  | loss: 2.04662 | val_0_unsup_loss_numpy: 3.8299500942230225|  0:00:00s
epoch 10 | loss: 1.87432 | val_0_unsup_loss_numpy: 2.237529993057251|  0:00:00s
epoch 11 | loss: 1.72491 | val_0_unsup_loss_numpy: 1.796429991722107|  0:00:00s
epoch 12 | loss: 1.63449 | val_0_unsu

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [19]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network

summary(tabnet_model, input_size=X_train.shape) 

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [1022, 39]                --
├─EmbeddingGenerator: 1-1                                    [1022, 39]                --
├─TabNetEncoder: 1-2                                         [1022, 16]                --
│    └─BatchNorm1d: 2-1                                      [1022, 39]                78
│    └─FeatTransformer: 2-2                                  [1022, 32]                4,352
│    │    └─GLU_Block: 3-1                                   [1022, 32]                4,800
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   -

In [20]:
encoder = tabnet_model.encoder
decoder = tabnet_model.decoder

In [21]:
class Sampling(nn.Module):
    def __init__(self, seed=1337):
        super(Sampling, self).__init__()
        self.seed = seed

    def forward(self, inputs):
        z_mean, z_log_var = inputs
        batch = z_mean.size(0)
        dim = z_mean.size(1)
        # #print(batch, dim)
        epsilon = torch.randn(batch, dim, generator=torch.Generator().manual_seed(self.seed))
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [22]:
class VAE_Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_Encoder, self).__init__()
        self.tabnet_encoder = tabnet_model.encoder
        self.mlp = nn.Sequential(
            nn.Linear(16, 128),  
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, latent_dim)
        )
        self.fc_mean = nn.Linear(latent_dim, latent_dim)
        self.fc_log_var = nn.Linear(latent_dim, latent_dim)
        self.sampling = Sampling()

    def forward(self, x):
        steps_output, _ = self.tabnet_encoder(x)
        encoded = steps_output[-1]
        # #print("Shape of encoded tensor:", encoded.shape)
        encoded = self.mlp(encoded)
        z_mean = self.fc_mean(encoded)
        z_log_var = self.fc_log_var(encoded)
        try:
            z = self.sampling((z_mean, z_log_var))
        except Exception as e:
            print(f"Error in sampling function: {e}")
            #print(f"z_mean shape: {z_mean.shape}")
            #print(f"z_log_var shape: {z_log_var.shape}")
        return z_mean, z_log_var, z


In [23]:
class VAE_Decoder(nn.Module):
    def __init__(self, latent_dim,encoded_dim, output_dim):
        super(VAE_Decoder, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim, 32),   
            nn.ReLU(),
            nn.Linear(32, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, encoded_dim),  
        )
        self.tabnet_decoder = tabnet_model.decoder
        self.reshape = nn.Unflatten(1, (encoded_dim,))
        self.output_dim=output_dim


    def forward(self, z):
        x = F.relu(self.mlp(z))

        # #print("Shape before reshape:", x.shape)
        # x = self.reshape(x)
        x = x[None, ...]

        # #print("Shape after reshape:", x.shape)
        # x = x.view(x.size(0), output_dim)
        
        output = self.tabnet_decoder(x)
        # #print(output.shape)
        # #print("Shape of output from tabnet_decoder:", output.shape)
        output = torch.sigmoid(output)
        output = output.view(-1, self.output_dim)
        return output

In [24]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from lightning.pytorch import LightningModule, Trainer
from torchmetrics import Accuracy, AUROC, F1Score, Precision, Recall
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

In [25]:
class VAE(LightningModule):
    def __init__(self, encoder, decoder, classifier, loss, learning_rate=1e-3):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.classifier = classifier
        self.learning_rate = learning_rate
        self.loss = loss

        self.val_accuracy = Accuracy(task="multiclass", num_classes=2)
        self.val_auroc = AUROC(task="multiclass", num_classes=2)
        self.val_f1 = F1Score(task="multiclass", num_classes=2)

    def forward(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        # #print("Shape after encoder:", z.shape)

        reconstruction = self.decoder(z)
        # #print("Shape after decoder:", reconstruction.shape)

        return reconstruction, z_mean, z_log_var, z

    def training_step(self, batch):
        # z_mean, z_log_var, z = self.encoder(data)
        # reconstruction = self.decoder(z)
        #print("Shape of data in training_step:", batch[0].shape)
        data, labels = batch
        reconstruction, z_mean, z_log_var, z = self.forward(data)
        #print("Shape of reconstruction in training_step:", reconstruction.shape)
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  # Tính tổng trên chiều (1, 2)
        )
        #print("Shape of reconstruction_loss:", reconstruction_loss.shape)
        preds_logit = self.classifier(z)
        #print("Shape of preds_logit in validation_step:", preds_logit.shape)
        #print("Shape of reconstruction in validation_step:", reconstruction.shape)
        #print("Shape of data in validation_step:", data.shape)
        #print("Shape of labels in validation_step:", labels.shape)
        classification_loss = self.loss(preds_logit, labels.long())
        probs = F.softmax(preds_logit, dim=1)
        #print("Shape of probs:", probs.shape)
        preds = torch.argmax(probs, dim=1)
        #print("Shape of preds after argmax:", preds.shape)
        #print(data.dtype, preds.dtype, probs.dtype)  # Chọn lớp có xác suất cao nhất


        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)
        kl_loss = torch.mean(torch.sum(kl_loss))
        vae_loss = reconstruction_loss + kl_loss
        total_loss = vae_loss + classification_loss

        self.log('train_loss', total_loss, prog_bar=True, on_step=True, on_epoch=True)
        return total_loss
    
    def validation_step(self, batch, batch_idx):
        data, labels = batch
        reconstruction, z_mean, z_log_var, z = self.forward(data)
        preds_logit = self.classifier(z)
        #print("Shape of preds_logit in validation_step:", preds_logit.shape)
        #print("Shape of reconstruction in validation_step:", reconstruction.shape)
        #print("Shape of data in validation_step:", data.shape)
        #print("Shape of labels in validation_step:", labels.shape)
        classification_loss = self.loss(preds_logit, labels.long())
        probs = F.softmax(preds_logit, dim=1)
        preds = torch.argmax(probs, dim=1)

        
        reconstruction, z_mean, z_log_var, z = self.forward(data)
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  # Tính tổng trên chiều (1, 2)
        )

        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)
        kl_loss = torch.mean(torch.sum(kl_loss))
        vae_loss = reconstruction_loss + kl_loss
        total_loss = vae_loss + classification_loss
        acc = self.val_accuracy(preds, labels)
        auroc = self.val_auroc(probs, labels)
        f1 = self.val_f1(preds, labels)

        # Logging
        self.log('val_loss', total_loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        self.log('val_auroc', auroc, prog_bar=True)
        self.log('val_f1', f1, prog_bar=True)

        return total_loss
     

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='max', factor=0.95, patience=10, min_lr=1e-6, verbose=True),
            'monitor': 'val_acc',
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

In [26]:
class SimpleClassifier(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, 64)  # Layer đầu tiên
        self.fc2 = nn.Linear(64, num_classes)  # Output layer: num_classes nodes

    def forward(self, z):
        x = F.relu(self.fc1(z))  # Activation ReLU
        x = self.fc2(x)          # Logits (chưa áp dụng softmax)
        return x

In [27]:
latent_dim = 64
encoded_dim = 16
output_dim = X_train.shape[1]
input_dim = X_train.shape[1]
#print(input_dim)

early_stopping = EarlyStopping(monitor='val_acc',
    # min_delta=0.00005,
    patience=60,
    verbose=1,
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    filename='best_model',
    save_top_k=1,
    mode='max',
)



In [28]:
vae_encoder = VAE_Encoder(latent_dim=latent_dim)
#print("Encoder Summary:")
summary(vae_encoder, input_size=(1022, input_dim))

Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Encoder                                                  [1022, 64]                --
├─TabNetEncoder: 1-1                                         [1022, 16]                --
│    └─BatchNorm1d: 2-1                                      [1022, 39]                78
│    └─FeatTransformer: 2-2                                  [1022, 32]                4,352
│    │    └─GLU_Block: 3-1                                   [1022, 32]                4,800
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   --                        (recursive)
│    └─ModuleList: 2-12                              

In [29]:
x = torch.randn(800, 39) 
steps_output, _ = tabnet_model.encoder(x)
encoded = steps_output[-1]
#print(f"Encoded shape: {encoded.shape}")

import torch

x = torch.randn(800, 39)  

steps_output, _ = tabnet_model.encoder(x)
#print("Shape of encoder output:", [output.shape for output in steps_output])

decoder_input = steps_output[-1] 
decoder_input = decoder_input[None, ...]
try:
    decoder_output = tabnet_model.decoder(decoder_input)
    #print(f"Decoder shape: {decoder_output.shape}")
except ValueError as e:
    print(f"Error: {e}")


In [30]:
vae_decoder = VAE_Decoder(latent_dim=latent_dim, encoded_dim=encoded_dim, output_dim=output_dim)
#print("Decoder Summary:")
summary(vae_decoder, input_size=(700, latent_dim), device="cpu")

Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Decoder                                                  [700, 39]                 --
├─Sequential: 1-1                                            [700, 16]                 --
│    └─Linear: 2-1                                           [700, 32]                 2,080
│    └─ReLU: 2-2                                             [700, 32]                 --
│    └─Linear: 2-3                                           [700, 96]                 3,168
│    └─ReLU: 2-4                                             [700, 96]                 --
│    └─Linear: 2-5                                           [700, 96]                 9,312
│    └─ReLU: 2-6                                             [700, 96]                 --
│    └─Linear: 2-7                                           [700, 16]                 1,552
├─TabNetDecoder: 1-2                                         [700, 39]             

In [ ]:
learning_rate=0.0001
classifier = SimpleClassifier(latent_dim, num_classes=2)
vae = VAE(encoder=vae_encoder, decoder=vae_decoder, classifier=classifier, loss=criterion, learning_rate=learning_rate)
summary(vae, input_size=(700, input_dim), device='cpu')

Layer (type:depth-idx)                                            Output Shape              Param #
VAE                                                               [700, 39]                 4,290
├─VAE_Encoder: 1-1                                                [700, 64]                 --
│    └─TabNetEncoder: 2-1                                         [700, 16]                 --
│    │    └─BatchNorm1d: 3-1                                      [700, 39]                 78
│    │    └─FeatTransformer: 3-2                                  [700, 32]                 9,152
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│  

: 

In [ ]:
trainer = Trainer(max_epochs=100, callbacks=[early_stopping, checkpoint_callback], accelerator="cpu", devices=1, enable_progress_bar=True, enable_model_summary=True, profiler="simple")
trainer.fit(vae, train_loader, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name         | Type               | Params | Mode 
------------------

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\lightning\pytorch\loops\fit_loop.py:310: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 16/16 [00:00<00:00, 26.76it/s, v_num=17, train_loss_step=0.623, val_loss=-3.02, val_acc=0.406, val_auroc=0.481, val_f1=0.406, train_loss_epoch=3.090]

Metric val_acc improved. New best score: 0.406


Epoch 1: 100%|██████████| 16/16 [00:00<00:00, 26.69it/s, v_num=17, train_loss_step=-4.09, val_loss=-6.83, val_acc=0.379, val_auroc=0.479, val_f1=0.379, train_loss_epoch=-1.22]

...

Epoch 6: 100%|██████████| 16/16 [00:00<00:00, 25.99it/s, v_num=17, train_loss_step=-15.0, val_loss=-18.1, val_acc=0.282, val_auroc=0.466, val_f1=0.282, train_loss_epoch=-14.1]

Metric val_acc improved by 0.006 >= min_delta = 0.0. New best score: 0.282


Epoch 7: 100%|██████████| 16/16 [00:00<00:00, 26.05it/s, v_num=17, train_loss_step=-15.0, val_loss=-19.4, val_acc=0.267, val_auroc=0.465, val_f1=0.267, train_loss_epoch=-15.3]

Metric val_acc improved by 0.015 >= min_delta = 0.0. New best score: 0.267


Epoch 34:   6%|▋         | 1/16 [00:00<00:00, 38.83it/s, v_num=17, train_loss_step=-17.4, val_loss=-23.1, val_acc=0.267, val_auroc=0.467, val_f1=0.267, train_loss_epoch=-19.2] 

In [ ]:
# learning_rate=0.0001
# optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
# num_epochs = 10
# for epoch in range(num_epochs):
#     vae.train()
#     train_loss = 0
#     for batch_data, _ in train_loader:
#         # #print(batch_data)
#         loss, rec_loss, kl_loss = vae.train_step(batch_data, optimizer)
#         train_loss += loss

#     train_loss /= len(train_loader)
#     #print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}")